In [1]:
%matplotlib inline
import re
import seaborn as sbn
import nltk
import tqdm as tqdm
import sqlite3
import pandas as pd
import numpy as np
nltk.download('stopwords')


import string
from nltk.corpus import stopwords
stop = stopwords.words("english")
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from math import floor,ceil
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

from sklearn.svm import LinearSVC

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sujaykumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('data/reviews_sports.csv')

/var/folders/l2/_mks1sd56yjdxq9514qt1fnh0000gn/T/ipykernel_7411/3608450076.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/reviews_sports.csv')


In [3]:
df = df.drop(['Unnamed: 0','style','vote','image'],axis=1)

<h2>As we are taking only verified Users we drop all the false Reviews</h2>

In [4]:
df["verified"].value_counts()

True     2611805
False     228135
Name: verified, dtype: int64

In [5]:
df = df[df['verified']]

In [6]:
df["verified"].value_counts()

True    2611805
Name: verified, dtype: int64

<h3>We Consider only the Text Based to Recommend the Products</h3>

In [7]:
df = df.drop(['reviewTime','reviewerName','unixReviewTime','verified'],axis=1)

In [8]:
df = df.dropna(axis=0)

In [9]:
df

,overall,reviewerID,asin,reviewText,summary
0,5.0,A180LQZBUWVOLF,0000032034,What a spectacular tutu! Very slimming.,Five Stars
1,1.0,ATMFGKU5SVEYY,0000032034,What the heck? Is this a tutu for nuns? I know...,Is this a tutu for nuns?!
2,5.0,A1QE70QBJ8U6ZG,0000032034,Exactly what we were looking for!,Five Stars
3,5.0,A22CP6Z73MZTYU,0000032034,I used this skirt for a Halloween costume and ...,I liked that the elastic waist didn't dig in (...
4,4.0,A22L28G8NRNLLN,0000032034,This is thick enough that you can't see throug...,This is thick enough that you can't see throug...
...,...,...,...,...,...
2839935,5.0,A3TN0U64HONOPB,B01HJGAJ9O,"Love the shorts.. amazing comfortable, perfect...",Five Stars
2839936,5.0,AJDQLM8PT3YWT,B01HJGAJ9O,Superb quality,Five Stars
2839937,5.0,A3QK5ZLRE2KHLL,B01HJHHBHG,Works every time,Quality catcher
2839938,5.0,A3VDML80KNR9QQ,B01HJHHBHG,I have a briley bolt release paddle installed ...,Works for A400 with Briley Bolt Release


In [10]:
df['overall'].value_counts()

5.0    1777587
4.0     450064
3.0     192538
1.0      98020
2.0      91986
Name: overall, dtype: int64

<h3> Cleaning the Data (NLP)</h3>

In [11]:
def data_clean(rev, remove_stopwords=True):
    if isinstance(rev, str):
        new_text = re.sub("[^a-zA-Z]", " ", rev)
        words = new_text.lower().split()
        if remove_stopwords:
            sts = set(stopwords.words("english"))
            words = [w for w in words if not w in sts]
        ary = []
        eng_stemmer = english_stemmer
        for word in words:
            ary.append(eng_stemmer.stem(word))
        return ary
    else:
        return []

In [ ]:
clean_reviewData = []
for rev in df['reviewText']:
    clean_reviewData.append( " ".join(data_clean(rev)))

clean_summaryData = []
for rev in df['summary']:
    clean_summaryData.append( " ".join(data_clean(rev)))

In [ ]:
Most_used_Words_Review = pd.Series(' '.join(clean_reviewData).lower().split()).value_counts()[:20]
print(Most_used_Words_Review)

In [ ]:
Most_used_Words_Summary = pd.Series(' '.join(clean_summaryData).lower().split()).value_counts()[:20]
print (Most_used_Words_Summary)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_vectorizer = TfidfVectorizer(min_df=4, max_features = 1000)
test_vecor = text_vectorizer.fit_transform(clean_reviewData)
tfidf_vector = dict(zip(text_vectorizer.get_feature_names_out(), text_vectorizer.idf_))

In [ ]:
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
sample_review = df.reviewText[:10]
for test in sample_review:
    test
    ss = analyser.polarity_scores(test)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]))
    print(test)

In [ ]:
test_reviewText = df.reviewText
test_Ratings = df.overall
text_vectorizer = TfidfVectorizer(max_df=.8)
text_vectorizer.fit(test_reviewText)
def rate(r):
    ary2 = []
    for rating in r:
        tv = [0,0,0,0,0]
        tv[rating-1] = 1
        ary2.append(tv)
    return np.array(ary2)

In [ ]:
test_reviewText

In [ ]:
test_Ratings

In [ ]:
df = pd.DataFrame({
    'reviewText': test_reviewText,
    'Ratings': test_Ratings
})

In [ ]:
df.to_csv("processed_data/final_pre-processed.csv")